In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab\ Notebooks/SC201L18
%ls

/content/drive/MyDrive/Colab Notebooks/SC201L18
RAG.ipynb  romeojuliet.txt


In [ ]:
%pip install langchain-community
%pip install langchain_openai
%pip install chromadb

In [ ]:
# Load File
from langchain.document_loaders import TextLoader
loader = TextLoader('romeojuliet.txt')
documents = loader.load()

In [ ]:
# Split Data Into Chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=40,
    length_function=len,
    add_start_index=True,
  )
chunks = text_splitter.split_documents(documents)

In [ ]:
# Set Up OpenAI API Key
import os
API_KEY = ''
os.environ['OPENAI_API_KEY'] = API_KEY

In [ ]:
# Data Embedding
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

CHROMA_PATH = 'my_database'

db = Chroma.from_documents(
    chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH,
)
db.persist()
print(f"Saved{len(chunks)} chunks to {CHROMA_PATH}.")

Saved194 chunks to my_database.


In [ ]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

embedding = OpenAIEmbeddings()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding)

query = input('Question: ')

results = db.similarity_search_with_relevance_scores(query, k=5)
if len(results) == 0 or results[0][1] < 0.7:
  print('Unable to find matching results')
else:
  context = '\n\n---\n\n'.join([doc.page_content for doc, score in results])
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context, question=query)

  model = ChatOpenAI()
  response = model.predict(prompt)

  sources = [doc.metadata.get("source", None) for doc, _score in results]
  formatted_response = f"Response: {response}\nSources: {sources}"
  print(formatted_response)



Question: Is Romeo's father nice?
Response: Based on the context provided, it is not clear if Romeo's father is nice or not.
Sources: ['romeojuliet.txt', 'romeojuliet.txt', 'romeojuliet.txt', 'romeojuliet.txt', 'romeojuliet.txt']
